In [ ]:
!pip install pm4py==2.2.31

In [ ]:
import pm4py
import pandas as pd
from itertools import product, permutations
from datetime import datetime
import graphviz

In [ ]:
data = [["A","C", "A", "B"],
        ["A","C", "A", "A"],
        ["A","C", "A", "D"],
        ["A","C", "A", "E"]]

In [ ]:
# data generatioln
l = [zip(w1,w2)
for w1 in permutations('ABCD') if w1.index("A") < w1.index("B") and w1.index("C") < w1.index("D")
for w2 in permutations('1234') if w2.index("1") < w2.index("2") and w2.index("3") < w2.index("4")  ]

data = [[x+y for (x,y) in z] for z in l ]
#data = [[x for (x,y) in z] for z in l ]

In [ ]:
# data generatioln
l = [zip(w1,w2)
for w1 in permutations('ABCD') if w1.index("A") < w1.index("B") and w1.index("C") < w1.index("D")
for w2 in permutations('1234')  ]

#data = [[x+y for (x,y) in z] for z in l ]
data = [[y for (x,y) in z] for z in l ]

In [ ]:
# data generatioln
l = [zip(w1,w2)
for w1 in permutations('ABCD') if w1.index("A") < w1.index("B") and w1.index("C") < w1.index("D")
for w2 in product('1234', repeat=4)  ]

data = [[x+y for (x,y) in z] for z in l ]
#data = [[y for (x,y) in z] for z in l ]

In [ ]:
# data to log
df = pd.DataFrame(columns=['case', 'activity', 'timestamp'])
for i, case in enumerate(data):
    for activity in case:
        df = df.append({'case': i, 'activity': activity, 'timestamp': datetime.now()}, ignore_index=True)
dataframe = pm4py.format_dataframe(df, case_id='case', activity_key='activity', timestamp_key='timestamp')
event_log = pm4py.convert_to_event_log(dataframe)

In [ ]:
# dfg extraction
dfg, start, end = pm4py.discover_directly_follows_graph(event_log)


In [ ]:
dot = graphviz.Digraph()
dot.graph_attr.update({'rankdir': 'LR'})
events = set([item for sublist in data for item in sublist])
for e in events:
  if e in start:
    dot.node(e, e, color="green")
  elif e in end:
    dot.node(e, e, color="red")
  else:
    dot.node(e, e)

for (orig, dest), freq in dfg.items():
  dot.edge(orig, dest, str(freq))
print(dot.source)

In [ ]:
process_tree = pm4py.discover_tree_inductive(event_log)
pm4py.view_process_tree(process_tree)

In [ ]:
# dfg reduction algorithm
import pm4py
import pandas as pd
from itertools import product, permutations
from datetime import datetime
import graphviz

# data generatioln
l = [zip(w1,w2)
for w1 in permutations('ABCD') if w1.index("A") < w1.index("B") and w1.index("C") < w1.index("D")
for w2 in permutations('1234') if w2.index("1") < w2.index("2") and w2.index("3") < w2.index("4")  ]

data = [[x+y for (x,y) in z] for z in l ]

# data to log
df = pd.DataFrame(columns=['case', 'activity', 'timestamp'])
for i, case in enumerate(data):
    for activity in case:
        df = df.append({'case': i, 'activity': activity, 'timestamp': datetime.now()}, ignore_index=True)
dataframe = pm4py.format_dataframe(df, case_id='case', activity_key='activity', timestamp_key='timestamp')
event_log = pm4py.convert_to_event_log(dataframe)


from collections import Counter
from copy import deepcopy

def get_reduced_edges(events, dfg, start, end):
  d = {}
  for e1 in events:
    for e2 in events:
      reduced_edges = 0
      if e1==e2 or (e2, e1) in d:
        continue
      if (e1 in start and not e2 in start) or (not e1 in start and e2 in start):
        continue
      if (e1 in end and not e2 in end) or (not e1 in end and e2 in end):
        continue
      if (e1,e2) in dfg:
        continue
      if (e2,e1) in dfg:
        continue
      for s in events:
        if (s, e1) in dfg and (s, e2) in dfg:
          reduced_edges += 1
        if (e1, s) in dfg and (e2, s) in dfg:
          reduced_edges += 1
      d[(e1,e2)] = reduced_edges
  return d

def merge_vertices(dfg, start, end, v1, v2):
  new_dfg, new_start, new_end = deepcopy(dfg), deepcopy(start), deepcopy(end)
  vertex = ",".join([v1,v2])
  if v1 in new_start and v2 in new_start:
    a = new_start.pop(v1)
    b = new_start.pop(v2)
    new_start[",".join([v1,v2])] = a + b
  if v1 in new_end and v2 in new_end:
    a = new_end.pop(v1)
    b = new_end.pop(v2)
    new_end[",".join([v1,v2])] = a + b
  for (orig, dest), freq in dfg.items():
    if v1 == orig or v2 == orig:
      new_dfg.pop((orig, dest))
      new_dfg[(vertex, dest)] = new_dfg.get((vertex, dest), 0) + freq
    if v1 == dest or v2 == dest:
      new_dfg.pop((orig, dest))
      new_dfg[(orig, vertex)] = new_dfg.get((vertex, dest), 0) + freq
  return new_dfg, new_start, new_end


events = set([item for sublist in data for item in sublist])
dfg, start, end = pm4py.discover_directly_follows_graph(event_log)
reduced_edges = get_reduced_edges(events, dfg, start, end)
while len(reduced_edges) != 0:
  v1, v2 = max(reduced_edges, key=reduced_edges.get)
  print(v1, v2)
  dfg, start, end = merge_vertices(dfg, start, end, v1, v2)
  events = [x for x in events if not x in [v1, v2]] + [",".join([v1,v2])]
  reduced_edges = get_reduced_edges(events, dfg, start, end)

dot = graphviz.Digraph()
dot.graph_attr.update({'rankdir': 'LR'})
for e in events:
  if e in start:
    dot.node(e, e, color="green")
  elif e in end:
    dot.node(e, e, color="red")
  else:
    dot.node(e, e)

for (orig, dest), freq in dfg.items():
  dot.edge(orig, dest, str(freq))
print(dot.source)

In [ ]:
# check that the grouping yields a good model
d = {}
for new_e in events:
  for e in new_e.split(","):
    d[e] = new_e
new_data = []
for word in data:
